# Construction of the pMEC9001, 2 and 3 vectors

The vector pMEC1049 vector was used in [Romaní et al. 2014](http://www.sciencedirect.com/science/article/pii/S096085241401757X) The pMEC1049 expresses a D-xylose metabolic pathway and has a hygromycin selectable marker. The details of the construction of pMEC1049 can be found [here](pMEC1049.ipynb).

This document describe the construction of the pMEC9001, 2 and 3 vectors. The pMEC9001 is the pMEC1049 with an additional expression cassette for the Saccharomyces cerevisiae gene HAA1(YPR008W). The pMEC9002 is the pMEC1049 with an additional expression cassette for S. cerevisiae PRS3(YHL011C) and pMEC9003 has both of them.

| Vector   | Relevant property                                            |
|----------|--------------------------------------------------------------|
| pMEC9001 | [HAA1](http://www.yeastgenome.org/locus/S000006212/overview) |
| pMEC9002 | [PRS3](http://www.yeastgenome.org/locus/S000001003/overview) |
| pMEC9003 | HAA1 & PRS3                                                  |

Normally, this would be done following the yeast pathway kit strategy by adding genes with a set of new promoters and terminators, but in this case a requirement was to retain the native promoters and terminators for HAA1 and PRS3.

The strategy involves linearizing the vector in two locations (before and after the xylose pathway) and adding the HAA1 and PRS3 expression cassettes amplified using tailed primers.


## The PRS3 construct

The PRS3 cassette was previously cloned according to the description below in vector YEpJCP according to this description:

"obtained by PCR amplification of fragment carrying PRS3 from Saccharomyces cerevisiae CEN.PK113-7D genomic DNA using appropriate primers and insertion into plasmid pGEM-T Easy. Cloning into YEplac195KanMX using EcoRI digestion sites." [Cunha et al. 2015](http://www.sciencedirect.com/science/article/pii/S0960852415006707)

Primers:

    P1: TTATCTTCATCACCGCCATAC
    P2: ACAAGAGAAACTTTTGGGTAAAATG

The exact same PRS3 fragment will be cloned in pMEC9002.

In [1]:
from pydna.parsers import parse_primers

In [2]:
p1,p2 = parse_primers('''
>P1
TTATCTTCATCACCGCCATAC
>P2
ACAAGAGAAACTTTTGGGTAAAATG
''')

We gain access to the S. cerevisiae genome through the [pygenome](https://pypi.python.org/pypi/pygenome) module.

In [3]:
from pygenome import saccharomyces_cerevisiae as sg

In [4]:
from pydna.dseqrecord import Dseqrecord

In [5]:
PRS3_locus = Dseqrecord(sg.stdgenes["PRS3"].locus())

The PRS3_locus contain the DNA from the end of the upstream ORF to the beginning of the downstream ORF.

In [6]:
PRS3_locus

Dseqrecord(-2963)

In [7]:
from pydna.amplify import pcr

In [8]:
PRS3_product = pcr(p1, p2, PRS3_locus)

In [9]:
PRS3_product

Amplicon(1546)

In [10]:
PRS3_product.figure()

5TTATCTTCATCACCGCCATAC...CATTTTACCCAAAAGTTTCTCTTGT3
                         |||||||||||||||||||||||||
                        3GTAAAATGGGTTTTCAAAGAGAACA5
5TTATCTTCATCACCGCCATAC3
 |||||||||||||||||||||
3AATAGAAGTAGTGGCGGTATG...GTAAAATGGGTTTTCAAAGAGAACA5

The primers anneal perfectly to the template, so this is the PCR product we want.

# HAA1 construct

We will now do the same with the HAA1 cassette.

Vector: BHUM1737

Construction: obtained by PCR amplification of a SalI/BamHI fragment carrying HAA1 from yeast genomic DNA using appropriate primers and subsequent insertion into plasmid YEplac195.

Primers described in [Malcher et al. 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3063667/) supporting information, Table S1

    HAA1hc_fw: GTC GAC CCC ATT TCC CCT TTC TTT TCC
    HAA1hc_rev: GGA TCC ATA CCT CAT CTC TGC GTG TTC G

In [11]:
from pydna.parsers import parse_primers

In [12]:
h1,h2 = parse_primers('''
>HAA1hc_fw
GTC GAC CCC ATT TCC CCT TTC TTT TCC
>HAA1hc_rev
GGA TCC ATA CCT CAT CTC TGC GTG TTC G
''')

In [13]:
HAA1_locus = Dseqrecord(sg.stdgenes["HAA1"].locus())

In [14]:
HAA1_locus

Dseqrecord(-4085)

In [15]:
HAA1_product = pcr(h1, h2, HAA1_locus)

In [16]:
HAA1_product

Amplicon(3227)

In [17]:
HAA1_product.figure()

      5CCCATTTCCCCTTTCTTTTCC...CGAACACGCAGAGATGAGGTAT3
                               ||||||||||||||||||||||
                              3GCTTGTGCGTCTCTACTCCATACCTAGG5
5GTCGACCCCATTTCCCCTTTCTTTTCC3
       |||||||||||||||||||||
      3GGGTAAAGGGGAAAGAAAAGG...GCTTGTGCGTCTCTACTCCATA5

These primers are tailed, but we have no reason to include these tails (containing restriction sites). We therefore cut six bp from the beginning and six bp from the end of the sequence:

In [18]:
h1 = h1[6:]
h2 = h2[6:]

In [19]:
HAA1_product = pcr(h1, h2, HAA1_locus)

In [20]:
HAA1_product.figure()

5CCCATTTCCCCTTTCTTTTCC...CGAACACGCAGAGATGAGGTAT3
                         ||||||||||||||||||||||
                        3GCTTGTGCGTCTCTACTCCATA5
5CCCATTTCCCCTTTCTTTTCC3
 |||||||||||||||||||||
3GGGTAAAGGGGAAAGAAAAGG...GCTTGTGCGTCTCTACTCCATA5

In [21]:
HAA1_product.seq

Dseq(-3215)
CCCA..GTAT
GGGT..CATA

Now we will have to design tailed primers for the HAA1_product and the PRS3_product sequences so that we can add them to pMEC1049 by gap repair. First we have to decide with which restriction enzymes we should open the pMEC1049 vector.

The restriction enzymes below are candidates for linearizing the pMEC1049 before an after the cassette.

[XhoI](http://rebase.neb.com/rebase/enz/XhoI.html) [AleI](http://rebase.neb.com/rebase/enz/AleI.html) [OliI](http://rebase.neb.com/rebase/enz/OliI.html)

These enzymes are also unique in the pYPK0 based vectors, so we can use tha same strategy to create vectors expressing only the pRS3 and/or HAA1 but without the xylose pathway if needed.

In [22]:
from pydna.readers import read

In [23]:
pMEC1049 = read("pMEC1049.gb")

In [24]:
pMEC1049

File(id)(o15488)

In [25]:
from Bio.Restriction import XhoI, AleI, OliI

In [26]:
pMEC1049_xho = pMEC1049.linearize(XhoI)

We design gap repair primers using the pydna assembly primers function

In [27]:
from pydna.design import assembly_fragments

In [28]:
fragments = assembly_fragments( [Dseqrecord(pMEC1049_xho.seq.mung()), HAA1_product, pMEC1049_xho] )

In [29]:
HAA1_product.seq

Dseq(-3215)
CCCA..GTAT
GGGT..CATA

In [30]:
Hfw = fragments[1].forward_primer
Hrv = fragments[1].reverse_primer

In [31]:
Hfw.id = "Hfw"
Hrv.id = "Hrv"

In [32]:
Hfw = Hfw[1:]
Hrv = Hrv[:-1]

In [33]:
Hfw = Hfw[:50] # we limit the length to 50 bp since these are less expensive from our provider
Hrv = Hrv[:50]

In [34]:
print( Hfw.format("tab") )

part_part_Hfw	GGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT



In [35]:
print( Hrv.format("tab") )

part_part_Hrv	AGACAAACCGTGGGACGAATTCTTAAGATGCTCGAATACCTCATCTCTGC



In [36]:
HAA1_recombination_product = pcr(Hfw, Hrv, HAA1_locus)

In [37]:
HAA1_recombination_product

Amplicon(3284)

In [38]:
HAA1_recombination_product.figure()

                                  5CCCATTTCCCCTTTCT...GCAGAGATGAGGTATT3
                                                      ||||||||||||||||
                                                     3CGTCTCTACTCCATAAGCTCGTAGAATTCTTAAGCAGGGTGCCAAACAGA5
5GGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT3
                                   ||||||||||||||||
                                  3GGGTAAAGGGGAAAGA...CGTCTCTACTCCATAA5

In [39]:
from pydna.assembly import Assembly

In [40]:
asm_haa1 = Assembly((pMEC1049_xho, HAA1_recombination_product))

In [41]:
asm_haa1

Assembly
fragments..: 15492bp 3284bp
limit(bp)..: 25
G.nodes....: 4
algorithm..: common_sub_strings

In [42]:
candidate = asm_haa1.assemble_circular()[0]

In [43]:
candidate.figure()

 -|pMEC1049_lin|34
|               \/
|               /\
|               34|3284bp_PCR_prod|35
|                                  \/
|                                  /\
|                                  35-
|                                     |
 -------------------------------------

In [44]:
pMEC9001 = candidate.synced(pMEC1049)

In [45]:
pMEC9001.stamp()

cSEGUID_BgBOlNv8lWGLExKGOpZa8J83H9I

In [46]:
pMEC9001.locus="pMEC9001"

The pMEC9001 is the pMEC1049 with HAA1. The sequence can be downloaded using the link below.

In [47]:
pMEC9001.write("pMEC9001.gb")

pMEC9001.gb

## PRS3

We will now make a pMEC1049 with PRS3 called pMEC9002.

In [48]:
pMEC1049_oli = pMEC1049.linearize(OliI)

The integration site was chosen to be the uniqie OliI site.

In [49]:
fragments2 = assembly_fragments((pMEC1049_oli, PRS3_product, pMEC1049_oli))

In [50]:
Pfw = fragments2[1].forward_primer
Prv = fragments2[1].reverse_primer

In [51]:
Pfw.id = "Pfw"
Prv.id = "Prv"

In [52]:
Prv=Prv[:-2]

In [53]:
Pfw = Pfw[:51]
Prv = Prv[:51]

In [54]:
print( Pfw.format("tab"))
print( Prv.format("tab"))

part_Pfw	TAACGATGTAGTACAGCGTTTCCGCTTTTTCACCCTTATCTTCATCACCGC

part_part_Prv	CATAAGTACCCATCCAAGAGCACGCTTATTCACCAACAAGAGAAACTTTTG



In [55]:
PRS3_recombination_product = pcr(Pfw, Prv, PRS3_locus)

In [56]:
PRS3_recombination_product

Amplicon(1616)

The recombination was designed for OliI but AleI was used.

In [57]:
pMEC1049_ale = pMEC1049.linearize(AleI)

In [58]:
asm_prs3 = Assembly((pMEC1049_ale, PRS3_recombination_product))

In [59]:
asm_prs3

Assembly
fragments..: 15488bp 1616bp
limit(bp)..: 25
G.nodes....: 4
algorithm..: common_sub_strings

In [60]:
candidate = asm_prs3.assemble_circular()[0]

In [61]:
candidate

Contig(o17034)

In [62]:
pMEC9002 = candidate.synced(pMEC1049)

In [63]:
pMEC9002.locus = "pMEC9002"

In [64]:
pMEC9002.stamp()

cSEGUID_5qldWd2jGxUyMnMKECLGpNvJv2M

The pMEC9002 vector is the pMEC1049 with PRS3

In [65]:
pMEC9002.write("pMEC9002.gb")

pMEC9002.gb

## pMEC9003

The HAA1 and PRS3 cassettes were added to the plasmid in one step to the plasmid digested with both XhoI and AleI. Cutting with XhoI and AleI makes two fragments about 6 and 9 kb.

In [66]:
pMEC1049_9kbp, pMEC1049_6kb = pMEC1049.cut(XhoI, AleI)

In [67]:
pMEC1049_6kb

Dseqrecord(-6017)

In [68]:
pMEC1049_9kbp

Dseqrecord(-9475)

In [69]:
pMEC1049_6kb.locus = "pMEC1049_6kb"
pMEC1049_9kbp.locus = "pMEC1049_9kbp"

In [70]:
asm_prs_haa = Assembly((pMEC1049_6kb, pMEC1049_9kbp, HAA1_recombination_product, PRS3_recombination_product))

In [71]:
asm_prs_haa

Assembly
fragments..: 6017bp 9475bp 3284bp 1616bp
limit(bp)..: 25
G.nodes....: 8
algorithm..: common_sub_strings

In [72]:
candidate = asm_prs_haa.assemble_circular()[0]

In [73]:
candidate

Contig(o20249)

In [74]:
pMEC9003 = candidate.synced(pMEC1049)

In [75]:
pMEC9003.stamp()

cSEGUID_hhymwO1hS1IXp9n4XX4eODGWuhU

In [76]:
pMEC9003.locus="pMEC9003"

pMEC9003 is the pMEC1049 with both HAA1 and PRS3. The sequence can be downloaded from the link below.

In [77]:
pMEC9003

Dseqrecord(o20249)

In [78]:
pMEC9003.write("pMEC9003.gb")

pMEC9003.gb